In [8]:
from collections import *

def train_char_lm(fname, order=4):
    f = open(fname,'r')
    data = f.read()
    lm = defaultdict(Counter)
#    pad = "~" * order
#    data = pad + data
    data = data.split('\n')
    for word in data:
        for i in range(len(word)-order):
            history, char = word[i:i+order], word[i+order]
            lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [2]:
from random import random

def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [29]:
from __future__ import print_function 

def hangman(lm, secret_word, guesser, max_mistakes=11, verbose=True):
    secret_word = secret_word.lower()
    n = len(secret_word)
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, n, lm)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            try:
                guessed.add(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes


In [96]:
def edit_data(fname,order):
    data = []
    f = open(fname)
    words = f.read().split('\n')
    for word in words:
        for c in ['e','i','a','o','u']:
            if c not in word:
                continue
            else:
                for char in word:
                    if char != c:
                        word = word[1:] + char
                    else:
                        break
                break
        word = '~'*(order) + word
        data.append(word)
    data = '\n'.join(data)
    f.close()
    file1 = open('edited'+fname,'w')
    file1.write(data)
    file1.close()

In [97]:
edit_data("corncob_lowercase.txt",order=4)

In [100]:
lm = train_char_lm("editedcorncob_lowercase.txt",order=4)

In [103]:
def maxLi(mask,guessed,nword,lm,order=4):
    if mask.count('_') == nword:
        for c in ['e','i','a','o','u']:
            if c not in guessed:
                return c
    else:
        for c in ['e','i','a','o','u']:
            if c in mask:
                break
        j = mask.index(c)
        i = j
        while i < len(mask):
            if mask[i] == '_':
                break
            else:
                i = i+1
#         temp = ''.join(mask).strip('_').split('_')[0]
        if i-j < order:
             hist = '~'*(order-i+j)+''.join(mask[j:i])
             print(hist)
        else:
             hist = ''.join(mask[i-order:i])
        c = generate_letter(lm,hist,len(hist))
        while c in guessed:
            c = generate_letter(lm,hist,len(hist))
        return c

In [ ]:
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [116]:
hangman(lm,'acrimoniously',maxLi)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ length 13
You have 11 attempts remaining.
Guess is e
Sorry, try again.
You have 10 attempts remaining.
Guess is i
Good guess: _ _ _ i _ _ _ i _ _ _ _ _
You have 10 attempts remaining.
~~~i
Guess is n
Good guess: _ _ _ i _ _ n i _ _ _ _ _
You have 10 attempts remaining.
~~~i
Guess is t
Sorry, try again.
You have 9 attempts remaining.
~~~i
Guess is u
Good guess: _ _ _ i _ _ n i _ u _ _ _
You have 9 attempts remaining.
~~~i
Guess is s
Good guess: _ _ _ i _ _ n i _ u s _ _
You have 9 attempts remaining.
~~~i
Guess is g
Sorry, try again.
You have 8 attempts remaining.
~~~i
Guess is a
Good guess: a _ _ i _ _ n i _ u s _ _
You have 8 attempts remaining.
~~~i
Guess is b
Sorry, try again.
You have 7 attempts remaining.
~~~i
Guess is r
Good guess: a _ r i _ _ n i _ u s _ _
You have 7 attempts remaining.
~~~i
Guess is d
Sorry, try again.
You have 6 attempts remaining.
~~~i
Guess is m
Good guess: a _ r i m _ n i _ u s _ _
You have 6 attemp

7

In [113]:
lm['ardw']

[('o', 0.8), ('a', 0.2)]

In [35]:
'___h___h'.strip('_').split('_')[0]

'h'

In [56]:
['w','h','a','t','e','v','e','r'].index('e')

4

In [95]:
'a' in 'what'

True